# ESA Trainee (Riga) - Mapping Crops and their Biophysical Characteristics with Polarimetric SAR and Optical Remote Sensing

Practical Wedneseday 29th June 2022: Crop specific time series analysis for growth monitoring

Prepared by UCLouvain-Geomatics

Adapted for the Agriculture Virtual Laboratory by Brockmann Consult

## Introduction & guidelines

This Jupyter-notebook allows to exploit the Sentinel-2 Leaf Area Index (LAI) time-series produced in the previous section of this training module using the SNAP software. The first step of this section corresponds to a quality control procedure and aims to detect and potentially remove any image showing outliers or marginal surface reflectance from the image time series. The second step investigates the respective temporal profiles of all corn fields of the study area, classifies their growing patterns over the season, and their corresponding intra-parcel heterogeneity.

## Environment and packages

This notebook runs without further package installation on the Agriculture Virtual Laboratory with any AVL user environment from version 1.1.5 onwards.

### Package importation

First the  required packages must be imported.

In [ ]:
import numpy as np
import pandas as pd
import geopandas
import rasterio
import plotly.express as px
from rasterio.plot import show
import glob, os, time
import folium
from collections import Counter
from datetime import datetime
import earthpy
import earthpy.spatial as es
import earthpy.plot as ep
import matplotlib.pyplot as plt
from shapely.geometry import box
from pyproj import Transformer
import contextily as cx
from shapely.geometry import Polygon, mapping
import plotly.graph_objects as go
from pathlib import Path
import warnings
import s3fs

print("All the packages are imported")

Define a `glob` function to find input data in object storage.

In [ ]:
s3filesystem = s3fs.S3FileSystem(anon=False)
def glob(globexpr: str) -> list[str]:
    return ['s3://' + path for path in s3filesystem.glob(globexpr)]

Define a directory to store working files and output data during notebook execution, and make sure that it exists. Data written to this directory can be viewed and downloaded in the JupyterLab environment.

In [ ]:
output_path = os.path.expanduser('~/arset-output/')
if not os.path.isdir(output_path):
    os.mkdir(output_path)

# 1. Importation of vector and raster data used in the notebook



## Preprocessing steps

All the Sentinel-2 images (2020 time series for the T31UFS tile) bands 2, 3, 4 and 8 (Blue, Green, Red and Near infrared) have been preprocessed using SNAP and cropped to the region of interest in order to reduce the computation time and the weight of the files included in this notebook. 

If the Sentinel-2 LAI images produced using SNAP as explained in the previous section are not available to you for any reason, you might use the so called ‘backup_training_data’ to continue with this Jupyter Notebook, found on the ARSET training page.


### Specification of the paths to the files used in this notebook

The ```work_path``` variable is the path where the folder containing the data is located. For the AVL version of the notebook, this variable is set to reference the required data in AVL’s object storage.

Once the required path is specified, a list of the paths to the single-bands tifs (red,green,blue and Near infrared bands) in the folder is created using the ```glob``` function.
The shapefile containing the parcels which will be analysed is also loaded in a geoDataFrame which is a data-structure similar to a pandas dataframe with an added geographic attribute.


In [ ]:
work_path = 's3://agriculture-vlab-data/ARSET/Training_data/'

vector_path = f"{work_path}shp"
bands_path  = f"{work_path}images"
lai_path    = f"{work_path}images"

blue_list  = sorted(glob(f"{bands_path}/*.data/B2.img"))
green_list = sorted(glob(f"{bands_path}/*.data/B3.img"))
red_list   = sorted(glob(f"{bands_path}/*.data/B4.img"))
nir_list   = sorted(glob(f"{bands_path}/*.data/B8.img"))
lai_list = sorted(glob(f"{bands_path}/*.data/lai.img"))


print(f'The image list of red bands contains {len(red_list)} images ')
print(f'The image list of green bands contains {len(green_list)} images ')
print(f'The image list of blue bands contains {len(blue_list)} images ')
print(f'The image list of near infra-red bands contains {len(nir_list)} images ')

invariant_shp_path = f"{vector_path}/invariant_shp/invariant_surfaces.shp"
invariant_gdf = geopandas.read_file(invariant_shp_path)
invariant_gdf = invariant_gdf.to_crs("epsg:32631")

print(f'The crs of the invariant surfaces shapefile is {invariant_gdf.crs}')


roi_gdf = geopandas.read_file(f"{vector_path}/crop_shp/all_crops_clipped_to_extent_english.shp")
roi_gdf = roi_gdf.set_crs('epsg:32631')
bands   = ['B2','B3','B4','B8']
#display(roi_gdf)
print(f'The crs of the maize shapefile is {roi_gdf.crs}')


# 2. Time series analysis: outlier detection

## 2.1 Visualization of the selected invariant targets (polygons) in the region of interest

The contextily package allows to display shapefiles overlaying basemaps

In [ ]:
fig, (ax,axs) = plt.subplots(1,2,figsize=(15, 15))

# Plot the invariant target on a basemap
ax.set_title("Visualization of the invariant target polygons", fontsize=15,y = 1)
invariant_gdf.plot(ax=ax, facecolor=None, edgecolor="white", alpha=1, lw=1, column='TYPE', legend = True, legend_kwds={'loc': 'upper left'})
cx.add_basemap(ax, source=cx.providers.Esri.WorldImagery, crs=invariant_gdf.crs.to_string())

# Create a bbox (=extent polygon) containing the invariant targets
extent_invariant = box(*invariant_gdf.geometry.total_bounds)
extent_invariant_gdf = geopandas.GeoDataFrame(index=[0], crs="epsg:32631", geometry=[extent_invariant])

# Plot the extent on  a basemap
axs.set_title('Shapefile of invariant surfaces extent')
extent_invariant_gdf.plot(ax = axs, facecolor="none", edgecolor="red", linewidth=4)
cx.add_basemap(axs, source=cx.providers.OpenStreetMap.Mapnik, crs=extent_invariant_gdf.crs.to_string())


plt.tight_layout()


## Visualization of surface reflectance images 

### Comparison of cloud-free RGB-image with a cloudy one 

Using the red, green, and blue bands of the images, true color composites can be plotted and clear and cloudy images can be compared. The earthpy library has a convenient function to do so and is used in the next cell.

In [ ]:

rgb_list_clear = [sorted(red_list)[5], sorted(green_list)[5], sorted(blue_list)[5]]

rgb_array = []

for image in rgb_list_clear:
    with rasterio.open(image) as src:
        out_image = src.read(1)
        rgb_array.append(out_image)
        
rgb_stack = np.array(rgb_array)

fig, (ax,axs) = plt.subplots(1,2,figsize=(15, 7))

ep.plot_rgb(
    rgb_stack,
    rgb=(0, 1, 2),
    ax=ax,
    title="Cloud-free Sentinel-2 RGB image clipped to the extent of the region of interest ",
    stretch=True,
    )

rgb_list_cloudy = [sorted(red_list)[1], sorted(green_list)[1], sorted(blue_list)[1]]
rgb_array = []

for image in rgb_list_cloudy:
    with rasterio.open(image) as src:
        out_image = src.read(1)
        rgb_array.append(out_image)
        
rgb_stack = np.array(rgb_array)


ep.plot_rgb(
    rgb_stack,
    rgb=(0, 1, 2),
    ax=axs,
    title="Cloudy Sentinel-2 RGB image clipped to the extent of the region of interest ",
    stretch=True,
)
plt.tight_layout()

## 2.2 Optional part: Use of Sen2Cor to mask clouds 

This section is for information purposes only and is intended to show how to mask clouds using sen2cor masks. The images used in this notebook are cloud-free and thus don't need to be masked.

## Creation of the folders that will contain the cloud-masked bands





In [ ]:
'''
for band in bands[:-1]:
    masked_path = f'{bands_path}_MASKED_SEN2COR_DA'
    if not os.path.isfile(masked_path):
        Path(masked_path).mkdir(parents=True, exist_ok=True)
        print(f'the masked bands folder of {band} has been created')
    else:
        print('the masked paths already exists')
'''

## Application of the Sen2Cor mask on the bands

The cloudy pixels are removed using the Masks from the cloud mask rasters. Here all the values in this raster that are not equal to 2,4,5,6 or 7 indicate that the corresponding pixels have to be masked. Each image of the mask list is thus reclassified to create binary rasters where 0 represents good pixels and 1 represents pixels to be masked out. 

The values of the mask are classes assigned by the algorithm and correspond to:

* 0 : No-data
* 1 : Saturated or defective pixel
* 2 : Dark area pixel
* 3 : Cloud shadows
* 4 : Vegetation
* 5 : Not vegetated
* 6 : Water
* 7 : Unclassified
* 8 : Cloud medium probability
* 9 : Cloud high probability
* 10 : Thin cirrus
* 11 : Snow

This new binary mask list is used to mask all the unusable pixels detected by Sen2Cors on each band of the rasters. 

In [ ]:
'''
nodata_val = -10000

for band in bands[:-1]:
    im_list = sorted(glob(f"{rasters_path}{band}/*.tif"))

    for im_file in im_list:

        # Get date of image
        date = os.path.basename(im_file)[7 : 7 + 9]

        # Find SCL corresponding to the given reflectances image
        scl_file = glob(f"{rasters_path}MASKS/*{date}.tif")[0]

        scl_file = scl_file.replace("\\", "/")

        im_file_scl = (
            f"{rasters_path}{band}_MASKED_SEN2COR_DA/{os.path.basename(im_file)[:-4]}_SCL.tif"
        )

        if not os.path.isfile(im_file_scl):

            # Open SCL and change invalid pixels categories by NaN
            src = rasterio.open(scl_file, "r")

            # Read file as numpy array
            SCL = src.read(1)
            src.close()

            # print('Scene Classification map')
            # show(SCL, cmap='Set3')

            SCL = SCL.astype(float)


            SCL[SCL == 0] = np.nan    
            SCL[SCL == 1] = np.nan    
            SCL[SCL == 2] = 1        
            SCL[SCL == 3] = np.nan    
            SCL[SCL == 4] = 1         
            SCL[SCL == 5] = 1         
            SCL[SCL == 6] = 1      
            SCL[SCL == 7] = 1      
            SCL[SCL == 8] = np.nan    
            SCL[SCL == 9] = np.nan    
            SCL[SCL == 10] = np.nan  
            SCL[SCL == 11] = np.nan   
            # Open file
            src = rasterio.open(im_file, "r")

            # Read file as numpy array
            im = src.read(1)

            # Update metadata
            profile = src.profile
            profile.update(
                dtype=rasterio.int16,  # Set to int16 it is lighter than float
                nodata=nodata_val,  # Set nodata value in metadata
                compress="lzw",
            )  # Compression option

            # Mask image reflectance with SCL
            im_SLC = im[0:1146,0:1270] * SCL

            # Change numpy NaN by nodata_val (e.g. -10000)
            im_SLC[np.isnan(im_SLC)] = nodata_val

            # Change the array's type : from float to integer 16
            im_SLC = im_SLC.astype(np.int16)

            # Write image
            dst = rasterio.open(im_file_scl, "w", **profile)
            dst.write(im_SLC, 1)

            # Close rasterio objects
            src.close()
            dst.close()

            print(f"A new raster file is created : {im_file_scl}")

    print("--> SCL is applied on all images !")
    '''

In [ ]:
'''
band = 'B4'

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))


im_file_clip = sorted(glob(f'{rasters_path}{band}/*.tif'))[8]

src = rasterio.open(im_file_clip, "r")
rasterio.plot.show(src, ax=ax1, cmap='Greys_r', title="Before masking the cloudy pixels")


im_file_scl = sorted(glob(f'{rasters_path}{band}_MASKED_SEN2COR/*.tif'))[8]

src = rasterio.open(im_file_scl, "r")

rasterio.plot.show(src, ax=ax2, cmap='Greys_r', title="After having masked the cloudy pixels")

plt.box(False)
'''



### Use of the folium library to display raster or vector data 

The folium library is designed to display geo-referenced data in an interactive way in the form of leaflets. Thanks to this type of display, different basemaps (OpenStreetMap, Google Satellite Imagery and plenty others)  can be used and several raster layers can be overlaid, activated or deactivated through the interface.

In the next cell, the first date of the time serie of the spectral band 4 (red) is displayed. The raster layer can be deactivated using the layer manager button on the top right corner of the leaflet.

### Vizualisation of the rasters using folium

In [ ]:
clipped_list = [sorted(red_list)[5], sorted(green_list)[5], sorted(blue_list)[5]]
bands = ["B2", "B3", "B4"]

m = folium.Map(
    [50.58634675562977, 5.038316875760848],
    zoom_start=10,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name = 'Satellite'
)
for band, name in zip(clipped_list, bands):
    with rasterio.open(band) as src:

        vmin, vmax = np.nanpercentile(src.read(1), (10,90 )) 
        im = src.read(1).astype(float)

        bounds = src.bounds
        bottom = bounds.bottom 
        left = bounds.left
        right = bounds.right
        top = bounds.top

        transformer = Transformer.from_crs("epsg:32631", "epsg:4326")

        x1, y1 = left, bottom
        x3, y3 = right, top

        x2, y2 = transformer.transform(x1, y1)
        x4, y4 = transformer.transform(x3, y3)

        im_loc = f"{output_path}ROI_example_{name}.png"

        ax.set_axis_off()

        plt.imsave(im_loc, im, vmin=vmin, vmax=vmax, cmap="Greys_r")

        img = folium.raster_layers.ImageOverlay(
            name=name,
            image=im_loc,
            bounds=[[x2-0.008, y2-0.006], [x4-0.008, y4-0.006]],
            opacity=1,
            interactive=True,
            cross_origin=False,
            zindex=1,
        )

        img.add_to(m)
folium.LayerControl().add_to(m)
m

## 2.3 Computation of the mean surface reflectance value for each date 

The mean reflectance of the pixels is computed for every invariant target polygon, every date, and the 4 bands. This will allow to assess the temporal consistency of the time series.

In [ ]:
geoms = invariant_gdf.geometry.values
geoms = [mapping(x) for x in geoms]
#band = 'B4'

band_array = []
for band in bands:
    stat_mat = []
    
    for im in sorted(glob(f"{bands_path}/*.data/{band}.img")):
        lai_vec = []
        for polygon in geoms:

            with rasterio.open(im) as src:

                out_image, out_transform = rasterio.mask.mask(
                    src, [polygon], crop=True, nodata=-10000
                )
                out_image = out_image.astype(float)
                out_image[(out_image > 55500.0) |(out_image == -10000.0)] = np.nan

                out_image = out_image/10000
                #print(np.nanmin(out_image),np.nanmean(out_image),np.nanmax(out_image))
                lai_vec.append(np.nanmean(out_image))
        stat_mat.append(lai_vec)
    print(f"The band {band} has been processed")
    band_array.append(stat_mat)
    

## 2.4 Count of the number of valid dates for each polygon

For each polygon, the number of dates where valid pixels can be used is counted. (Here the images are cloud free and not masked so all the dates will be valid, in other contexts, no-data values could make certain dates unusable.)

In [ ]:
for j in range(len(invariant_gdf["geometry"])):
    type = np.array(invariant_gdf["TYPE"])[j]
    polygon = [i[j] for i in band_array[2]]
    cnt = np.count_nonzero(np.isnan(polygon))
    print(
        f"There are {cnt} non valid dates and {len(polygon)-cnt} valid dates for the polygon of the {type} "
    )
    

## 2.5 Plot of the surface reflectance time series for the blue, green and red bands and outlier detection


 The time series of surface reflectance in the red band from pixels of the 5 different land cover types are plotted.

This section will focus on the analysis of the time series of each polygon. To detect the outliers, the median of the time series will first be calculated. Then, a threshold deviation from this median will be defined, and any date whose reflectance for a given pixel exceeds the threshold will be transformed into no data. 

The defined deviation for this notebook is 0.01, meaning that all values of the time series for each polygon exceeding the sum of the median and the deviation or being smaller than the difference between the median and the deviation are considered as an invalid date. 


The variable ```outliers_dates_mat``` will store the dates detected as outliers 

In [ ]:
outliers_dates_mat = []

## Time series analysis of the blue band

In [ ]:
dates = [datetime.strptime(x[-46:-38],'%Y%m%d' )for x in blue_list]


threshold = 0.015


fig, axs = plt.subplots(3, 2, figsize=(15, 10))
types = invariant_gdf["TYPE"]
for j, k, l in zip(
    range(len(invariant_gdf["geometry"])), [axs[0, 1], axs[0, 0], axs[1, 1], axs[1, 0],axs[2,0]], types
):

    ts = [i[j] for i in band_array[0]]
    median = [np.nanmean(ts)]*len(ts)
    max_dev = [i + threshold for i in median]
    min_dev = [i - threshold for i in median]

    outliers = [[count,i] for count,i in enumerate(ts) if i >= max_dev[0] or i <= min_dev[0]]
    outliers_dates = [dates[i[0]].strftime("%d/%m/%Y") for i in outliers]



    markers_on = np.argwhere(~np.isnan(ts))
    markers_on = [i[0] for i in markers_on]

    if len(outliers) > 0 and l!= 'quarry' and l != 'warehouse roof':
        print(f'there are {len(outliers)} outliers detected on the invariant surface {l}')    
        print(f'Outliers dates : {outliers_dates}')
        outliers_dates_mat.append(outliers_dates)
        
    k.plot(dates, ts, "bD", markevery=markers_on)
    k.plot(dates,median,  linestyle='--',color = 'green',label = 'Median reflectance')
    k.plot(dates,max_dev,  linestyle='--',color = 'black',label = 'Median + threshold')
    k.plot(dates,min_dev,  linestyle='-.',color = 'black',label = 'Median - threshold')

    
    k.set_ylim()
    k.title.set_text(
        f"{str(l)}"
    )
    k.grid(True)
    k.legend()
    fig.suptitle('Time series of the mean surface \n reflectance in the blue band of \n 5 types of polygons',fontweight="bold",y = 1.1)
    axs[2,1].set_visible(False)
    plt.tight_layout()
    

## Time series analysis of the green band

In [ ]:
dates = [datetime.strptime(x[-46:-38],'%Y%m%d' )for x in green_list]




fig, axs = plt.subplots(3, 2, figsize=(15, 10))
types = invariant_gdf["TYPE"]
for j, k, l in zip(
    range(len(invariant_gdf["geometry"])), [axs[0, 1], axs[0, 0], axs[1, 1], axs[1, 0],axs[2,0]], types
):

    ts = [i[j] for i in band_array[1]]
    
    median = [np.nanmean(ts)]*len(ts)
    max_dev = [i + threshold for i in median]
    min_dev = [i - threshold for i in median]
    outliers = [[count,i] for count,i in enumerate(ts) if i >= max_dev[0] or i <= min_dev[0]]
    outliers_dates = [dates[i[0]].strftime("%d/%m/%Y") for i in outliers]


    if len(outliers) > 0 and l!= 'quarry' and l != 'warehouse roof':
        print(f'there are {len(outliers)} outliers detected on the invariant surface {l}')    
        print(f'Outliers dates : {outliers_dates}')
        outliers_dates_mat.append(outliers_dates)

    markers_on = np.argwhere(~np.isnan(ts))
    markers_on = [i[0] for i in markers_on]
    
    k.plot(dates, ts, "gD", markevery=markers_on)
    k.plot(dates,median,  linestyle='--',color = 'green',label = 'Median reflectance')
    k.plot(dates,max_dev,  linestyle='--',color = 'black',label = 'Median + threshold')
    k.plot(dates,min_dev,  linestyle='-.',color = 'black',label = 'Median - threshold')



    k.set_ylim()
    k.title.set_text(
        f"{str(l)}"
    )
    k.grid(True)
    k.legend()
    fig.suptitle('Time series of the mean surface \n reflectance in the green band of \n 5 types of polygons',fontweight="bold",y = 1.1)
    axs[2,1].set_visible(False)
    plt.tight_layout()
    

## Time series analysis of the red band

In [ ]:
dates = [datetime.strptime(x[-46:-38],'%Y%m%d' )for x in red_list]


threshold = 0.015
fig, axs = plt.subplots(3, 2, figsize=(15, 10))
types = invariant_gdf["TYPE"]
for j, k, l in zip(
    range(len(invariant_gdf["geometry"])), [axs[0, 1], axs[0, 0], axs[1, 1], axs[1, 0],axs[2,0]], types
):

    ts = [i[j] for i in band_array[2]]
    median = [np.nanmean(ts)]*len(ts)
    max_dev = [i + threshold for i in median]
    min_dev = [i - threshold for i in median]
    outliers = [[count,i] for count,i in enumerate(ts) if i >= max_dev[0] or i <= min_dev[0]]
    outliers_dates = [dates[i[0]].strftime("%d/%m/%Y") for i in outliers]
    
    if len(outliers) > 0 and l!= 'quarry' and l != 'warehouse roof':
        print(f'there are {len(outliers)} outliers detected on the invariant surface {l}')    
        print(f'Outliers dates : {outliers_dates}')
        #outliers_dates_mat.append(outliers_dates)


    markers_on = np.argwhere(~np.isnan(ts))
    markers_on = [i[0] for i in markers_on]
    k.plot(dates, ts, "rD", markevery=markers_on)
    k.plot(dates,median,  linestyle='--',color = 'green',label = 'Median reflectance')
    k.plot(dates,max_dev,  linestyle='--',color = 'black',label = 'Median + threshold')
    k.plot(dates,min_dev,  linestyle='-.',color = 'black',label = 'Median - threshold')



    k.set_ylim()
    k.title.set_text(
        f"{str(l)}"
    )
    k.grid(True)
    k.legend()
    fig.suptitle('Time series of the mean surface \n reflectance in the red band of \n 5 types of polygons',fontweight="bold",y = 1.15)
    axs[2,1].set_visible(False)
    plt.tight_layout()


In [ ]:
flat_list = [item for sublist in outliers_dates_mat for item in sublist]
count =  Counter(flat_list)
count = count.most_common()

for i in count:
    print(f'the date {i[0]} has been detected as an outlier {i[1]} time(s) over invariants surfaces and spectral bands')
    

# 3. LAI time series analysis: intra- and inter-field crop growth quality assessment

## 3.1 Vizualisation 

### The extent of the shapefile containing the corn parcels is computed to display the area of interest

This extent can also be used to clip images and reduce computation time.

In [ ]:
fig,ax =plt.subplots(figsize = (20,20))

extent = box(*roi_gdf.geometry.total_bounds)
extent_gdf = geopandas.GeoDataFrame(index=[0], crs="epsg:32631", geometry=[extent])

extent_gdf.plot(ax = ax, facecolor="none", edgecolor="red", linewidth=4)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, crs=roi_gdf.crs.to_string())
ax.set_title('Shapefile extent')

#extent_gdf.to_file(filename= f"{work_path}extent.shp")


### Computation of LAI value ranges for the time series

The single-band rasters are plotted, slightly augmenting the contrast for visualization purposes. 

In [ ]:
im_list = [lai_list[15],lai_list[16],lai_list[20]]
print(f'the image list contains the following images : {[os.path.basename(i) for i in im_list]}')

print(f'the dates of the image list are : {np.sort(dates)[15]}, {np.sort(dates)[16]} and {np.sort(dates)[20]}')

In [ ]:

bands = ["LAI_1", "LAI_2", "LAI_3"]

m = folium.Map(
    [50.58634675562977, 5.038316875760848],
    zoom_start=10,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name = 'Satellite'
)
for band, name in zip(im_list, bands):
    with rasterio.open(band) as src:

        vmin, vmax = np.nanpercentile(src.read(1), (6, 94)) 
        im = src.read(1).astype(float)

        bounds = src.bounds
        bottom = bounds.bottom 
        left = bounds.left
        right = bounds.right
        top = bounds.top

        transformer = Transformer.from_crs("epsg:32631", "epsg:4326")

        x1, y1 = left, bottom
        x3, y3 = right, top

        x2, y2 = transformer.transform(x1, y1)
        x4, y4 = transformer.transform(x3, y3)

        im_loc = f"{output_path}ROI_example_{name}.png"

        ax.set_axis_off()

        plt.imsave(im_loc, im, vmin=vmin, vmax=vmax, cmap="Greys_r")

        img = folium.raster_layers.ImageOverlay(
            name=name,
            image=im_loc,
            bounds=[[x2-0.008, y2-0.006], [x4-0.008, y4-0.006]],
            opacity=1,
            interactive=True,
            cross_origin=False,
            zindex=1,
        )

        img.add_to(m)
folium.LayerControl().add_to(m)
m

### Visualization of the extent of the region of interest on a LAI multi-temporal color composite

In [ ]:
rgb_list = im_list[0:3]
for i in rgb_list:
    print(f'The band {i[-6:]} can be added to the composite')
rgb_array = []
for image in rgb_list:
    with rasterio.open(image) as src:
        out_image, out_transform = rasterio.mask.mask(
            src, extent_gdf.geometry, all_touched=True, crop=True
        )

        rgb_array.append(out_image)

rgb_stack = np.vstack(rgb_array)


fig, ax = plt.subplots(figsize=(15, 7))
ep.plot_rgb(
    rgb_stack,
    rgb=(2, 1, 0),
    ax=ax,
    title="LAI multi-temporal color composite at 10m : 15th May (Red), 1st June (Green), 14 September (Blue)",
    stretch=True,
)
plt.tight_layout()
plt.show()

### Visualization of the shapefiles

In [ ]:
maize_gdf = roi_gdf[roi_gdf['CROP_NAME'] == 'Maize (for livestock)']

print(f'There are {len(maize_gdf)} maize parcels in the shapefile')
maize_gdf = maize_gdf.sample(frac = 0.20,random_state = 1)
print(f'There are {len(maize_gdf)} selected maize parcels ')


fig, axs = plt.subplots( figsize=(20, 20))

axs.set_title("Visualization of the parcels", fontsize=25)


maize_gdf.plot(
    ax=axs,
    facecolor=None,
    edgecolor="red",
    alpha=0.8,
    lw=3
)

cx.add_basemap(axs, source=cx.providers.Esri.WorldImagery, crs=roi_gdf.crs.to_string())

plt.tight_layout()

# 3.2 Computation of the mean LAI value at parcel level for each date of the time series

The first step is to load the raster containing the cloud-free LAI time series. 
Then the computation of the mean LAI by date and by parcel takes place in four successive steps: 

1. First the polygons of all parcels are stored in a variable
2. For each polygon, a mask is created and used to mask all the other pixels than the ones included in the polygon
3. Finally, the mean LAI on this parcel is computed for each date and the result is stored in a vector (here ```lai_vec```)
4. The vectors (```lai_vec```) storing the LAI time series of each parcel are stored in a matrix (```lai_mat```) which will later be converted do a table (here called a pandas Dataframe)

This step can take some time to compute 

In [ ]:
LAI_raster_list = sorted(glob(f"{lai_path}/*.data/lai.img"))
stacked_raster_array,transform  = es.stack(LAI_raster_list)
stacked_raster_path = f'{output_path}/stacked_LAI.tif'


In [ ]:
with rasterio.open(stacked_raster_path, 'w', **transform) as dst:
    dst.write(stacked_raster_array)
    
geoms = maize_gdf.geometry.values

geoms = [mapping(x) for x in geoms]

lai_mat = []
for parcel in geoms:
    with rasterio.open(f'{output_path}/stacked_LAI.tif') as src:
        lai_vec = []
        out_image, out_transform = rasterio.mask.mask(src, [parcel], crop=True, nodata=-32768)
        
        out_image = np.where(out_image==-32768, np.nan, out_image)
        for date in range(np.shape(out_image)[0]):
            
            lai_vec.append(np.nanmean(out_image[date, :, :]))
            
        lai_mat.append(lai_vec)
print(f'The matrix contains {len(lai_mat)} time series')

### Creation of a dataframe containing the time series computed above
Once the mean LAI is computed for each parcel and for each date, the data is stored in a dataframe.


In [ ]:
df = pd.DataFrame(lai_mat)
lai_df = df.transpose()

In [ ]:
print(f'The first image of the list is {os.path.basename(sorted(glob(f"{lai_path}/*.data/lai.img"))[0])}')
dates = [datetime.strptime(x[-47:-39],'%Y%m%d' )for x in sorted(glob(f"{lai_path}/*.data/lai.img"))]

The indexes of the dataframe are set as the dates of each images and the column names as the parcel IDs.

In [ ]:
lai_df.index = dates
date_to_remove = lai_df.index[-2]

In [ ]:
lai_df.index = dates
lai_df.columns = maize_gdf.index
lai_unsorted = lai_df
lai_df = lai_df.sort_index()

lai_df = lai_df.drop(date_to_remove)

display(lai_df)


The plotly library is used to plot the LAI time series of all the parcels. The mean, 25th percentile, and 75th percentile are also added to the plot. 
This library can be used in a more interactive way than matplotlib and allows to look at individual profiles. 
* Double clicking on any element of the legend removes all the lines on the plot, and clicking on single legend elements makes them visible again. 
* Plotly also allows to make screenshots and zoom on particular areas of the plot using the menu on the top-right corner of the plot. 

In [ ]:
fig = go.Figure() 

for col,i in zip(lai_df.columns,range(len(lai_df.columns))):
    fig.add_trace(go.Scatter(x=lai_df.index, y=lai_df[col].values,
                                 name = col,
                                 mode = 'lines'))
    
fig.add_trace(go.Scatter(x = lai_df.index,y = lai_df.median(axis=1),line_color = 'black', line = dict(width=4),name = 'mean'))
fig.add_trace(go.Scatter(x = lai_df.index,y = lai_df.quantile(q= 0.25,axis=1),line_color = 'firebrick',  line = dict(width=4),name = '25-percentile'))
fig.add_trace(go.Scatter(x = lai_df.index,y = lai_df.quantile(q= 0.75,axis=1),line_color = 'firebrick', line = dict(width=4),name = '75-percentile'))
fig.update_layout(legend_traceorder="reversed")


fig.update_layout(height=800, width=800,title = 'LAI time series of all the Maize parcels in the area of interest')
fig.show()


### Focus on the corn growth cycle 
In order to consider the growth cycle of the corn sowed early May, and harvested in September–October, the beginning date of the time series will be set to the first of May and the end date to the 15th of November.

The dataframe is also displayed, each column representing a corn parcel and each row a date.

In [ ]:
lai_df_reduced = lai_df[(lai_df.index > '2020-05-01') & (lai_df.index < '2020-11-15')]
dates_before = len(lai_df[(lai_df.index <= '2020-05-01') ])
display(lai_df[(lai_df.index <= '2020-05-01') ])
fig = go.Figure() 

for col,i in zip(lai_df_reduced.columns,range(len(lai_df_reduced.columns))):
    fig.add_trace(go.Scatter(x=lai_df_reduced.index, y=lai_df_reduced[col].values,
                                 name = col,
                                 mode = 'lines'))
    
fig.add_trace(go.Scatter(x = lai_df_reduced.index,y = lai_df_reduced.mean(axis=1),line_color = 'black', line = dict(width=4),name = 'mean'))
fig.add_trace(go.Scatter(x = lai_df_reduced.index,y = lai_df_reduced.quantile(q= 0.25,axis=1),line_color = 'firebrick',  line = dict(width=4),name = '25-percentile'))
fig.add_trace(go.Scatter(x = lai_df_reduced.index,y = lai_df_reduced.quantile(q= 0.75,axis=1),line_color = 'firebrick', line = dict(width=4),name = '75-percentile'))
fig.update_layout(legend_traceorder="reversed")


fig.update_layout(height=800, width=800,title = 'LAI time series of all the Maize parcels in the area of interest between may and november')
fig.show()


## 3.3 Inter-field analysis of the LAI temporal profile
The average maize profile corresponds here to the normal crop growth condition.  

* First, the area under each LAI curve is computed for the whole season to quantify the crop growth condition in one single metric. 
* Second, for all the maize parcels of the region of interest, the distribution of these integrated LAI is then presented and quantiles are calculated.
* Third, 5 categories are defined based on quantiles (i.e. percentile between 15 and 20 ,percentile 40, percentile 60, percentile 80). Those categories are displayed on the histogram and correspond respectively to very poor, poor, medium, good, and very good growth conditions. Based on the above graph, temporal profiles hardly corresponding to regular corn crops (e.g. very low value until August) should not be considered for the inter-field analysis and are assumed to belong to the lowest percentiles (0-15).
* Finally, parcels are classified according to these categories and this information is written as an attribute in a new shapefile to examine using QGIS. 

The trapezoidal integretion method is used to compute the area under each curve (auc) which are stored in the ```auc``` variables

In [ ]:
auc_vec = [(np.trapz(parcel_timeserie)) for parcel_timeserie in lai_df_reduced.transpose().values.tolist()]
auc_df = pd.DataFrame(auc_vec)

auc = [np.array(x)[0] for x in np.array(auc_df)]
print(auc)

In [ ]:
auc_df.columns = ['Area Under Curve']
fig = px.histogram(auc_vec,nbins = 8,title = 'Histogram of the areas under the curves computed for all maize parcels')
fig.show()

lai_df_reduced_transposed = lai_df_reduced.transpose()
auc_df.index = lai_df_reduced_transposed.index

display(auc_df)
#display(lai_df_reduced_transposed)

### Preparation of 5 dataframes containing the time series of each category (very low, low, medium, high, very high)

The dataframes will later be concatenated to obtain a single table containing the time series and their crop-growth quality assessment computed a few cells earlier.
The 'very low' dataframe is displayed as an example.

In [ ]:
warnings.filterwarnings("ignore") #

very_low_df = lai_df_reduced_transposed[(auc_df['Area Under Curve']> np.nanpercentile(auc,15)) & (auc_df['Area Under Curve'] < np.nanpercentile(auc,20))] 
very_low_df['Quality']= 'very low'
low_df = lai_df_reduced_transposed[(auc_df['Area Under Curve'] > np.nanpercentile(auc,20)) &(auc_df['Area Under Curve'] < np.nanpercentile(auc,40))] 
low_df['Quality']= 'low'

medium_df = lai_df_reduced_transposed[(auc_df['Area Under Curve'] > np.nanpercentile(auc,40)) &(auc_df['Area Under Curve'] < np.nanpercentile(auc,60))]
medium_df['Quality']= 'medium'

high_df = lai_df_reduced_transposed[(auc_df['Area Under Curve'] > np.nanpercentile(auc,60)) &(auc_df['Area Under Curve'] < np.nanpercentile(auc,80))]
high_df['Quality']= 'high'
very_high_df = lai_df_reduced_transposed[(auc_df['Area Under Curve'] > np.nanpercentile(auc,80)) &(auc_df['Area Under Curve'] < np.nanpercentile(auc,100))] 
very_high_df['Quality']= 'very high'

display(very_low_df.head())


### Concatenation of the dataframes into one dataframe in which each parcel (columns) has row values corresponding to the mean LAI at each date and one  quality assessment attribute as last row.

In [ ]:
concatenation = pd.concat([medium_df, very_high_df,high_df,low_df,very_low_df])
final_df = concatenation.transpose()
#display(final_df.head())

Here the time series for each parcel are plotted with a line-color varying with the quality assessment: 
 
 |color|Qality|
 |---|---|
 |Red lines | very low |
 |Orange lines| low  |
 |Yellow lines | medium  |
 | Green lines | high  |
 |Blue lines | very high  |


In [ ]:
cmap = {'very low':'red','low':'orange','medium':'yellow','high':'green','very high':'blue'} 
colors = [cmap[x] for x in final_df.transpose()["Quality"]]

fig = go.Figure()
for col,i in zip(final_df.columns,range(len(final_df.columns))):
    fig.add_trace(go.Scatter(x=final_df.index[:-1], y=final_df[col].values[:-1],
                                 name = col,
                                 mode = 'lines',
                                 line_color=colors[i]))
fig.update_layout(height=800, width=800,title = 'Time-filtered time series with colors according to the crop growth quality assessment')
fig.show()

### Table join using indices between the final dataframe containing the information regarding the assessment of crop growth quality and the partial Geodataframe containing the parcels.


Before the table join: 

In [ ]:
to_join = final_df.transpose()['Quality']
display(to_join)
display(maize_gdf)

After the table join: 

In [ ]:
new_gdf = roi_gdf.filter(items = to_join.index, axis=0)
new_gdf['Quality'] = to_join
display(new_gdf)

### Save of the file as a shapefile that can be further explored in QGIS

In [ ]:
'''
new_gdf_path = f'{work_path}/maize_LAI_quality_assessment.shp'

if not os.path.isfile(new_gdf_path):
    new_gdf.to_file(new_gdf_path)
else:
    print('the shp already exists')
'''

# 3.4 Intra-field variability analysis at the peak of the LAI on a given parcel

A subset of parcels containing 3 parcels of very-low, medium, and very high parcels is first selected. Then, the date where the LAI is at its highest point is computed for each parcel. The parcels are then plotted on their maximal LAI day for visual inspection.

In [ ]:
very_low_subset = new_gdf[new_gdf['Quality']=='very low'].iloc[0:3,:]
medium_subset = new_gdf[new_gdf['Quality']=='medium'].iloc[[4,7,8],:]
very_high_subset = new_gdf[new_gdf['Quality']=='very high'].iloc[[0,2,4],:]

total_subset = pd.concat([very_low_subset,medium_subset,very_high_subset])

In [ ]:
subset = geopandas.GeoDataFrame(lai_df_reduced.filter(items = total_subset.index, axis=1))
max_values = subset.idxmax(axis=0).values
print(max_values)
indexes = pd.DataFrame(subset).idxmax(axis=0)
positions = [lai_unsorted.index.get_loc(i) for i in indexes]
print(positions)
display(subset)


In [ ]:
print(f'the max LAI of the first very-low labelled parcel is obtained at the date  {str(max_values[0])[:10]} of the time series')
print(f'the max LAI of the first medium labelled parcel is obtained at the date  {str(max_values[1])[:10]} of the time series')
print(f'the max LAI of the first very-high labelled parcel is obtained at the date  {str(max_values[2])[:10]} of the time series')

### Plot of three parcels of the very-low, medium, and very high categories 

In [ ]:
fig,axs = plt.subplots(3,3,figsize = (10,10),gridspec_kw={'width_ratios': [1, 1, 1]})
geoms = total_subset.geometry.values
geoms = [mapping(x) for x in geoms]
cols = ['Very low \n quality parcels','Medium \n quality parcels','Very high \n quality parcels']
axes = [axs[0,0],axs[1,0],axs[2,0],axs[0,1],axs[1,1],axs[2,1],axs[0,2],axs[1,2],axs[2,2]]
for parcel,date,i,j in zip(geoms,positions,axes,range(9)):
    with rasterio.open(stacked_raster_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, [parcel], crop=True, nodata=-32768)
        out_image = np.where(out_image==-32768, np.nan, out_image)
        output = out_image[date, :, :]
        im = i.imshow(output,cmap = 'BrBG',vmin = 0,vmax = 6)
        fig.colorbar(im,ax = i,fraction=0.046, pad=0.04,label = 'LAI' )
        i.set_title(f'parcel {subset.columns[j]}', y=1.05, fontsize=18)
        
for ax, col in zip(axs[0], cols):
    ax.annotate(col, xy=(0.5, 1.25), xytext=(0, 15),
                xycoords='axes fraction', textcoords='offset points',
                size='large', ha='center', va='baseline')

fig.subplots_adjust(hspace=0.5, bottom=0.1)
fig.suptitle('Plot of three parcels of very-low, medium and very high categories',y = 1.15)
plt.tight_layout()
